## 1) 라이브러리 + 데이터셋 전처리

In [92]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [93]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [94]:
# 컬럼 정의 + 데이터 로드

header_names = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
    "wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised",
    "root_shell","su_attempted","num_root","num_file_creations","num_shells",
    "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
    "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",
    "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count",
    "dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
    "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate",
    "attack_type","difficulty"
]

train_df = pd.read_csv("NSL-KDD_datasets/nsl-kdd/KDDTrain+.txt", names=header_names)
test_df  = pd.read_csv("NSL-KDD_datasets/nsl-kdd/KDDTest+.txt",  names=header_names)

In [95]:
# attack_type -> attack_category 매핑

dos = {"back","land","neptune","pod","smurf","teardrop","apache2","udpstorm","processtable","mailbomb"}
probe = {"satan","ipsweep","nmap","portsweep","mscan","saint"}
r2l = {"guess_passwd","ftp_write","imap","phf","multihop","warezmaster","warezclient",
       "spy","xlock","xsnoop","snmpguess","snmpgetattack","httptunnel","sendmail","named"}
u2r = {"buffer_overflow","loadmodule","perl","rootkit","ps","sqlattack","xterm"}

def map_attack(x):
    if x in ["normal","benign"]: return "benign"
    if x in dos: return "dos"
    if x in probe: return "probe"
    if x in r2l: return "r2l"
    if x in u2r: return "u2r"
    return "attack"

train_df["attack_category"] = train_df["attack_type"].apply(map_attack)
test_df["attack_category"]  = test_df["attack_type"].apply(map_attack)

In [96]:
# 불필요한 컬럼 제거

drop_cols = ["attack_type", "difficulty", "num_outbound_cmds"]

train_df.drop(columns=drop_cols, errors="ignore", inplace=True)
test_df.drop(columns=drop_cols, errors="ignore", inplace=True)

In [97]:
# X, y 분리

y_train_raw = train_df["attack_category"]
y_test_raw  = test_df["attack_category"]

X_train_raw = train_df.drop(columns=["attack_category"])
X_test_raw  = test_df.drop(columns=["attack_category"])

In [98]:
# 원-핫 인코딩 (train/test 컬럼 정렬)

all_X = pd.concat([X_train_raw, X_test_raw], axis=0)

cat_cols = all_X.select_dtypes(include=["object"]).columns
all_X = pd.get_dummies(all_X, columns=cat_cols, drop_first=True)

X_train = all_X.iloc[:len(X_train_raw)]
X_test  = all_X.iloc[len(X_train_raw):]

In [99]:
# 스케일링

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [100]:
# 라벨 인코딩

le = LabelEncoder()
le.fit(pd.concat([y_train_raw, y_test_raw], axis=0))

y_train = le.transform(y_train_raw).astype(np.int64)
y_test  = le.transform(y_test_raw).astype(np.int64)

num_classes = len(le.classes_)
print("classes:", le.classes_)

classes: ['attack' 'benign' 'dos' 'probe' 'r2l' 'u2r']


In [101]:
# LSTM 입력 형태로 변환

X_train_lstm = X_train.astype(np.float32).reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm  = X_test.astype(np.float32).reshape(X_test.shape[0],  1, X_test.shape[1])

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

print("X_train_lstm:", X_train_lstm.shape)
print("X_test_lstm :", X_test_lstm.shape)

X_train_lstm: (125973, 1, 118)
X_test_lstm : (22544, 1, 118)


## 2) Dataset / DataLoader

In [102]:
class NslKddDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)  # float32
        self.y = torch.from_numpy(y)  # int64

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 256

train_ds = NslKddDataset(X_train_lstm, y_train)
test_ds  = NslKddDataset(X_test_lstm,  y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, drop_last=False)

## 3) LSTM 분류 모델 정의 (최소 구조)

In [103]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (B, T, F)
        out, (hn, cn) = self.lstm(x)
        # hn: (num_layers, B, hidden_size) -> 마지막 레이어의 hidden 사용
        h_last = hn[-1]                 # (B, hidden_size)
        h_last = self.dropout(h_last)
        logits = self.fc(h_last)        # (B, num_classes)
        return logits

## 4) 불균형 대응: class_weight로 loss 가중치 주기 (SMOTE 대신 추천)

In [104]:
classes_in_train = np.unique(y_train)  # y_train에 실제로 존재하는 라벨만
w = compute_class_weight(
    class_weight="balanced",
    classes=classes_in_train,
    y=y_train
).astype(np.float32)

# CrossEntropyLoss는 weight 길이가 num_classes여야 하므로 전체 길이로 확장
w_full = np.ones(num_classes, dtype=np.float32)
w_full[classes_in_train] = w

class_weights_t = torch.tensor(w_full, dtype=torch.float32)

In [105]:
num_features = X_train_lstm.shape[2]   # (N, 1, F)에서 F

print(num_features)

118


## 5) 학습 루프 (GPU 자동 사용)

In [106]:
model = LSTMClassifier(
    input_size=num_features,
    hidden_size=64,
    num_classes=num_classes,
    num_layers=1,
    dropout=0.2
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

criterion = nn.CrossEntropyLoss(weight=class_weights_t.to(device))

def train_one_epoch(model, loader):
    model.train()
    total_loss, total_correct, total = 0.0, 0, 0

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * Xb.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == yb).sum().item()
        total += Xb.size(0)

    return total_loss / total, total_correct / total

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_correct, total = 0.0, 0, 0
    all_preds, all_y = [], []

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        logits = model(Xb)
        loss = criterion(logits, yb)

        total_loss += loss.item() * Xb.size(0)
        preds = torch.argmax(logits, dim=1)

        total_correct += (preds == yb).sum().item()
        total += Xb.size(0)

        all_preds.append(preds.cpu().numpy())
        all_y.append(yb.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_y = np.concatenate(all_y)
    return total_loss / total, total_correct / total, all_y, all_preds

In [107]:
epochs = 15
best_val_acc = 0.0
best_state = None

for epoch in range(1, epochs + 1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader)
    te_loss, te_acc, y_true, y_pred = evaluate(model, test_loader)

    print(f"[{epoch:02d}/{epochs}] "
          f"train loss={tr_loss:.4f}, acc={tr_acc:.4f} | "
          f"test loss={te_loss:.4f}, acc={te_acc:.4f}")

    if te_acc > best_val_acc:
        best_val_acc = te_acc
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

print("Best test acc:", best_val_acc)

[01/15] train loss=0.4889, acc=0.8959 | test loss=1.9033, acc=0.7751
[02/15] train loss=0.1810, acc=0.9521 | test loss=2.2809, acc=0.7743
[03/15] train loss=0.1453, acc=0.9607 | test loss=2.5622, acc=0.7652
[04/15] train loss=0.1207, acc=0.9640 | test loss=2.9657, acc=0.7681
[05/15] train loss=0.1096, acc=0.9709 | test loss=3.1874, acc=0.7691
[06/15] train loss=0.0963, acc=0.9750 | test loss=3.2989, acc=0.7833
[07/15] train loss=0.0836, acc=0.9755 | test loss=3.4060, acc=0.7904
[08/15] train loss=0.0802, acc=0.9778 | test loss=3.7224, acc=0.7937
[09/15] train loss=0.0763, acc=0.9776 | test loss=3.8438, acc=0.7931
[10/15] train loss=0.0721, acc=0.9791 | test loss=3.8121, acc=0.7975
[11/15] train loss=0.0655, acc=0.9793 | test loss=4.0407, acc=0.7981
[12/15] train loss=0.0640, acc=0.9804 | test loss=4.2263, acc=0.7979
[13/15] train loss=0.0642, acc=0.9797 | test loss=4.3041, acc=0.7956
[14/15] train loss=0.0601, acc=0.9813 | test loss=4.4911, acc=0.7942
[15/15] train loss=0.0609, acc=0.9

## 6) 최종 평가 (리포트 + 혼동행렬)

In [108]:
# best 모델 복원
if best_state is not None:
    model.load_state_dict(best_state)

_, _, y_true, y_pred = evaluate(model, test_loader)

print(classification_report(y_true, y_pred, target_names=le.classes_))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

      attack       0.00      0.00      0.00         2
      benign       0.73      0.96      0.83      9711
         dos       0.96      0.86      0.90      7458
       probe       0.84      0.74      0.78      2421
         r2l       0.90      0.14      0.25      2885
         u2r       0.09      0.64      0.16        67

    accuracy                           0.80     22544
   macro avg       0.59      0.56      0.49     22544
weighted avg       0.84      0.80      0.77     22544

Confusion matrix:
 [[   0    0    0    0    0    2]
 [   0 9371   57  238   10   35]
 [   0 1033 6383   41    0    1]
 [   0  374  232 1780   29    6]
 [   0 2038    7   57  415  368]
 [   0   15    4    0    5   43]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  